In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

dataSet = pd.read_csv('heartCopy.csv')
dataSet_dimension = 12

for col in dataSet.select_dtypes(include="object"):
    encoder = LabelEncoder()
    dataSet[col] = encoder.fit_transform(dataSet[col])

scaler = MinMaxScaler()
dataSet_scaled = pd.DataFrame(scaler.fit_transform(dataSet), columns=dataSet.columns)

X = dataSet_scaled.drop(columns=["HeartDisease"]).to_numpy()
y = dataSet_scaled["HeartDisease"].to_numpy()


feature_train, feature_test, label_train, label_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Standard

from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

dataSet_feature_map = ZZFeatureMap(feature_dimension=dataSet_dimension, reps=2, entanglement="linear")
print(dataSet_feature_map)

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

dataSet_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=dataSet_feature_map)

In [ ]:
#Trial 1
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

def custom_feature_map(num_features, reps):

    features = ParameterVector('x', num_features)
    qc = QuantumCircuit(num_features)
    
    for a in range(reps):
        for i in range(num_features):
            qc.rx(features[i], i)
            qc.rz(features[i]**2, i)

        for i in range(num_features - 1):
            qc.cx(i, i + 1)
    
    return qc

dataSet_feature_map = custom_feature_map(dataSet_dimension - 1, reps=4)
print(dataSet_feature_map)

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

dataSet_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=dataSet_feature_map)

In [ ]:
#Trial 2
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

def custom_feature_map(num_features, reps):

    features = ParameterVector('x', num_features)
    qc = QuantumCircuit(num_features)
    
    for a in range(reps):
        for i in range(num_features):
            qc.rx(features[i]*2, i)
            qc.rz((features[i])**3, i)

        for i in range(num_features - 1):
            qc.cx(i, i + 1)
    
    return qc


dataSet_feature_map = custom_feature_map(dataSet_dimension - 1, reps=4)
print(dataSet_feature_map)

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

dataSet_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=dataSet_feature_map)

In [7]:
#Trial 3
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

def custom_feature_map(num_features, reps):

    features = ParameterVector('x', num_features)
    qc = QuantumCircuit(num_features)
    
    for a in range(reps):
        for i in range(num_features):
            qc.rx(features[i], i)
            qc.rz((features[i])**3, i)

        for i in range(num_features - 1):
            qc.cx(i, i + 1)
    return qc


dataSet_feature_map = custom_feature_map(dataSet_dimension - 1, reps=4)
print(dataSet_feature_map)

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

dataSet_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=dataSet_feature_map)

       ┌──────────┐┌─────────────┐      ┌──────────┐┌─────────────┐»
 q_0: ─┤ Rx(x[0]) ├┤ Rz(x[0]**3) ├───■──┤ Rx(x[0]) ├┤ Rz(x[0]**3) ├»
       ├──────────┤├─────────────┤ ┌─┴─┐└──────────┘└─┬──────────┬┘»
 q_1: ─┤ Rx(x[1]) ├┤ Rz(x[1]**3) ├─┤ X ├─────■────────┤ Rx(x[1]) ├─»
       ├──────────┤├─────────────┤ └───┘   ┌─┴─┐      └──────────┘ »
 q_2: ─┤ Rx(x[2]) ├┤ Rz(x[2]**3) ├─────────┤ X ├───────────■───────»
       ├──────────┤├─────────────┤         └───┘         ┌─┴─┐     »
 q_3: ─┤ Rx(x[3]) ├┤ Rz(x[3]**3) ├───────────────────────┤ X ├─────»
       ├──────────┤├─────────────┤                       └───┘     »
 q_4: ─┤ Rx(x[4]) ├┤ Rz(x[4]**3) ├─────────────────────────────────»
       ├──────────┤├─────────────┤                                 »
 q_5: ─┤ Rx(x[5]) ├┤ Rz(x[5]**3) ├─────────────────────────────────»
       ├──────────┤├─────────────┤                                 »
 q_6: ─┤ Rx(x[6]) ├┤ Rz(x[6]**3) ├─────────────────────────────────»
       ├──────────┤├─────────────┤

In [ ]:
from sklearn.svm import SVC

dataSet_svc = SVC(kernel=dataSet_kernel.evaluate)

dataSet_svc.fit(feature_train, label_train)

df_score_callable_function = dataSet_svc.score(feature_test, label_test)

print(f"Callable kernel classification test score: {df_score_callable_function}")